<text>
    <h1 style="text-align:center"> <a href="https://archive.ics.uci.edu/ml/datasets/Computer+Hardware">  Computer Hardware project  </a>  </h1>
    <br>
    <h3>Documentation: </h3>
    <ul> 
        <li> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html">  MinMaxScaler  </a>  </li>   
        <li> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html">  Linear Regression</a> </li> 
        <li> <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.kernel_ridge">  Ridge Regression</a> </li> 
        <li> <a href="https://www.w3schools.com/python/pandas/default.asp">  Pandas </a>  </li> 
        
</text>

<text>
    For this project, given a set of data containing comuter hardaware we will train a machine to give the best results.
    Some categories(columns) may provide unbalance data which will not be useful for our machine which will determine us to drop the column.
    At the end we will train our machine and use Linear Regression and Ridge Regression for proper results.
</text>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics         import confusion_matrix
from sklearn.metrics         import mean_absolute_error as mae 
from sklearn.metrics         import mean_squared_error  as mse
from sklearn.neighbors       import KNeighborsClassifier as knn
from sklearn.linear_model    import LinearRegression, Ridge, RidgeCV

In [2]:
def interpret (model, x_train, y_train, x_test, y_test, label = None, fit=True):
    # this function helps us to interpret the performances of the model by showing the matrics and the ConfMatrix  

    if fit:
        try: model.fit(x_train, y_train)
        except: pass

    if label: print (label + ' Interpretation:')

    predicted_train = model.predict(x_train)
    predicted_test  = model.predict(x_test)

    print(f'Train Score: {model.score(x_train, y_train) :.3f}') 
    print(f'Train mae:   {mae(predicted_train, y_train) :.3f}')
    print(f'Train mse:   {mse(predicted_train, y_train) :.3f}')

    print()

    print(f'Test  Score: {model.score(x_test, y_test) :.3f}') 
    print(f'Test  mae:   {mae(predicted_test, y_test) :.3f}')
    print(f'Test  mse:   {mse(predicted_test, y_test) :.3f}')

    print()

<h3>
    Preprocesing
</h3>
<text>
     In the following sequences we are dropping the irrelevant information, preventing the missclasification. 
     <br>
     Following the previous action , we will normalise the data in order to favorise the clasification.
</text>

In [3]:
dataset = pd.read_csv('machine.data', sep=',', header=None, names = ['vendor', 'model', 'myct', 'nmin', 'nmax', 'cach', 'chmin', 'chmax', 'prp', 'erp']) \
    .drop(columns=['model', 'vendor'])
display(dataset.head(10))

# unbalanced input -> droped
# keys = dataset['vendor'].value_counts().keys()
# vendors = dict(zip(keys, list(range(len(keys)))))
# dataset['vendor'] = dataset['vendor'].apply(lambda x: vendors[x])

# all unique models -> droped
# keys = dataset['model'].value_counts().keys()
# models = dict(zip(keys, list(range(len(keys)))))
# dataset['model'] = dataset['model'].apply(lambda x: models[x])

display(dataset.head(10))
display(dataset.describe())

,myct,nmin,nmax,cach,chmin,chmax,prp,erp
0,125,256,6000,256,16,128,198,199
1,29,8000,32000,32,8,32,269,253
2,29,8000,32000,32,8,32,220,253
3,29,8000,32000,32,8,32,172,253
4,29,8000,16000,32,8,16,132,132
5,26,8000,32000,64,8,32,318,290
6,23,16000,32000,64,16,32,367,381
7,23,16000,32000,64,16,32,489,381
8,23,16000,64000,64,16,32,636,749
9,23,32000,64000,128,32,64,1144,1238


,myct,nmin,nmax,cach,chmin,chmax,prp,erp
0,125,256,6000,256,16,128,198,199
1,29,8000,32000,32,8,32,269,253
2,29,8000,32000,32,8,32,220,253
3,29,8000,32000,32,8,32,172,253
4,29,8000,16000,32,8,16,132,132
5,26,8000,32000,64,8,32,318,290
6,23,16000,32000,64,16,32,367,381
7,23,16000,32000,64,16,32,489,381
8,23,16000,64000,64,16,32,636,749
9,23,32000,64000,128,32,64,1144,1238


,myct,nmin,nmax,cach,chmin,chmax,prp,erp
count,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000
mean,203.822967,2867.980861,11796.153110,25.205742,4.698565,18.267943,105.622010,99.330144
std,260.262926,3878.742758,11726.564377,40.628722,6.816274,25.997318,160.830733,154.757102
min,17.000000,64.000000,64.000000,0.000000,0.000000,0.000000,6.000000,15.000000
25%,50.000000,768.000000,4000.000000,0.000000,1.000000,5.000000,27.000000,28.000000
50%,110.000000,2000.000000,8000.000000,8.000000,2.000000,8.000000,50.000000,45.000000
75%,225.000000,4000.000000,16000.000000,32.000000,6.000000,24.000000,113.000000,101.000000
max,1500.000000,32000.000000,64000.000000,256.000000,52.000000,176.000000,1150.000000,1238.000000


<h4>
    DataSet
</h4>
<text>
    We are spliting the data into input and output.
</text>

In [4]:
train, test = train_test_split(dataset, test_size=0.1)
y_train = train['erp'].to_numpy().astype(np.int)
y_test = test['erp'].to_numpy().astype(np.int)
x_train = train.drop(columns=['erp']).to_numpy()
x_test = test.drop(columns=['erp']).to_numpy()

<h4>
    Interpretation using Linear Regression and Ridge Regression.
</h4>

In [5]:
interpret(LinearRegression(), x_train, y_train, x_test, y_test)

Train Score: 0.960
Train mae:   20.326
Train mse:   1008.339

Test  Score: 0.947
Test  mae:   20.355
Test  mse:   633.005



In [6]:
interpret(Ridge(), x_train, y_train, x_test, y_test)

Train Score: 0.960
Train mae:   20.326
Train mse:   1008.339

Test  Score: 0.947
Test  mae:   20.355
Test  mse:   633.004



In [7]:
interpret(RidgeCV(), x_train, y_train, x_test, y_test)

Train Score: 0.960
Train mae:   20.326
Train mse:   1008.339

Test  Score: 0.947
Test  mae:   20.354
Test  mse:   633.002

